In [1]:
!pip install huggingface_hub pydantic dataset datasets

In [ ]:
!pip install "distilabel[hf-transformers, openai]>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [3]:
pip install distilabel[outlines]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.2 MB/s eta 0:00:00


In [5]:
from huggingface_hub import get_token
from huggingface_hub import InferenceClient
from datasets import load_dataset
from toolz import take
from datasets import get_dataset_config_names
import torch, json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from pydantic import Field
from distilabel.steps import Step, StepInput
from distilabel.steps.typing import StepOutput
from typing import List
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub, KeepColumns, LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration
from distilabel.llms import TransformersLLM

In [6]:
class GeneratePrompts(Step):

    @property
    def inputs(self) -> List[str]:
        # Specify the input fields expected by this step
        return ['text']

    @property
    def outputs(self) -> List[str]:
        # Specify the output fields that this step will produce
        return ['text', 'instruction']

    def process(self, inputs: StepInput) -> StepOutput:

        for example in inputs:
          instruction = f"""
            Let's write abstract descriptions of sentences. Example:
            Sentence: Pilate's role in the events leading to the crucifixion lent themselves to melodrama , even tragedy , and Pilate often has a role in medieval mystery plays .
            Description: A description of a historical religious figure's involvement in a significant event and its later portrayal in art.
            Note: Descriptions can differ in the level of abstraction, granularity and the part of the sentence they focus on. Some descriptions need to be abstract, while others should be concrete and detailed.
            For the following sentence, write up 5 good and stand-alone, independent descriptions and 5 bad descriptions (which may be related, but are clearly wrong). Output a json file with keys 'good', 'bad'.
            Sentence: {example['text']}
            Start your answer with a curly bracket.
            """
          example['instruction'] = instruction

        yield inputs

In [24]:
class ParseDescriptions(Step):

    @property
    def inputs(self) -> List[str]:
        # Specify the input fields expected by this step
        return ['descriptions']

    @property
    def outputs(self) -> List[str]:
        # Specify the output fields that this step will produce
        return ['good_description', "bad_description"]

    def process(self, inputs: StepInput) -> StepOutput:

        new_generations = []
        for example in inputs:
          try:
            parsed = json.loads(example['descriptions'])
            for good, bad in zip(parsed['good'], parsed['bad']):
              new_generations.append({
                'text': example['text'],
                'instruction': example['instruction'],
                'good_description': good,
                'bad_description': bad
              })
          except:
            print("Can't parse descriptions")
            print(example['descriptions'])

        yield inputs

In [4]:
from pydantic import BaseModel

class Description(BaseModel):
    good: list[str]
    bad: list[str]

In [25]:
with Pipeline(name='sentence-similarity-pipeline') as synthesis_pipeline:
    load_data_from_hub = LoadDataFromHub(
        name='load-data-from-hub',
    )

    generate_prompts = GeneratePrompts(
        name='generate-prompts',
    )

    synthesizer = TextGeneration(
        name='description-text-genration',
        llm = TransformersLLM(
            model="microsoft/Phi-3.5-mini-instruct",
            device="cuda:0",
            structured_output={"format": "json", "schema": Description}
        ),
        output_mappings={"generation": "descriptions"}
    )

    parse_descriptions = ParseDescriptions(
        name='parse-descriptions',
    )

    load_data_from_hub >> generate_prompts >> synthesizer >> parse_descriptions


In [26]:
distiset = synthesis_pipeline.run(
    parameters={
        load_data_from_hub.name: {
            'repo_id': "ahsanirfan961/title-content-dataset",
            "split": "train"
        },
        synthesizer.name: {
            "llm": {
                "generation_kwargs": {
                    "max_new_tokens": 1024,
                    "temperature": 0.7,
                },
            },
        }
    },
    use_cache=False
)

[09/14/24 15:07:30] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=561280;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=324786;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#696\696]8;;\
                             '/root/.cache/distilabel/pipelines/sentence-similarity-pipeline/8e0022e364            
                             f6ca81185b93db4e6fa0625df2e593/data'                                                  

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in     ]8;id=857396;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=123143;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#705\705]8;;\
                             stages:                                                                               
                              * Stage 0: ['load-data-from-hub', 'generate-prompts',                                
                             'description-text-genration', 'parse-descriptions']                                   

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to load... ]8;id=224201;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=208203;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#918\918]8;;\

[09/14/24 15:07:33] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 3/4                  ]8;id=633637;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=230078;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#950\950]8;;\
                              * 'load-data-from-hub' replicas: 1/1                                                 
                              * 'generate-prompts' replicas: 1/1                                                   
                              * 'description-text-genration' replicas: 0/1                                         
                              * 'parse-descriptions' replicas: 1/1                                                 

...

                    INFO     ['distilabel.step.description-text-genration'] 🏁 Finished running ]8;id=404061;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=346522;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#127\127]8;;\
                             step 'description-text-genration' (replica ID: 0)                                     

                    INFO     ['distilabel.step.parse-descriptions'] 📦 Processing batch 0 in    ]8;id=720099;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=288688;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#217\217]8;;\
                             'parse-descriptions' (replica ID: 0)                                                  

                    INFO     ['distilabel.step.parse-descriptions'] 📨 Step                     ]8;id=272939;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=481934;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#274\274]8;;\
                             'parse-descriptions' sending batch 0 to output queue                                  

                    INFO     ['distilabel.step.parse-descriptions'] 🏁 Finished running step    ]8;id=489442;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=124556;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#127\127]8;;\
                             'parse-descriptions' (replica ID: 0)                                                  

Generating train split: 0 examples [00:00, ? examples/s]

In [27]:
print(distiset)

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['title', 'text', 'instruction', 'descriptions', 'distilabel_metadata', 'model_name', 'good_description', 'bad_description'],
            num_rows: 9
        })
    })
})


In [28]:
print(distiset['default']['train'].to_pandas())

                                               title  \
0  U.S. Navy plane crashes in Philippine Sea, thr...   
1  Zimbabwe's Mnangagwa arrives home, to be sworn...   
2  Cleveland officials to outline security for Re...   
3  U.S. senator calls for GAO probe to protect ba...   
4  Republican ideas for healthcare reforms could ...   
5  White House says 'couple of dozen' people stil...   
6  Democratic leaders Schumer, Pelosi to dine wit...   
7  Famine survey warns of thousands dying daily i...   
8  Qatar calls Trump's Jerusalem move 'death sent...   

                                                text  \
0  a us navy transport plane carrying  people cra...   
1  zimbabwe s former vice president emmerson mnan...   
2  the mayor and police chief of cleveland next w...   
3  the ranking democrat on the us senate subcommi...   
4  president donald trump’s push to fulfill a cam...   
5  the white house on sunday defended its impleme...   
6  us president donald trump invited senate dem

In [23]:
print(distiset['default']['train'].to_pandas()['generation'][3])

{ "good": [ "A political representative seeks oversight into how well US states adhere to laws aimed at safeguarding infants exposed to drugs via maternal use.", ", An inquiry regarding improved implementation measures within Congressional framework concerning infant safety amidst parental narcotic misuse cases." ], "bad": ["An analysis detailing economic impact studies linked to urban development policies.", "Description focused solely on culinary advancements introduced between two distinct geopolitical entities."] }
